In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "simple_white"

%cd ..
from models.cconv import CConvModel
from datasets.density_data_module import DensityDataModule
from utils.train_helper import load_hparams

/home/jakob/Bachelorarbeit/GNNDensityGradients
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [134]:

# Default network 32k
checkpoint  = "/home/jakob/Bachelorarbeit/GNNDensityGradients/lightning_logs/anova/default1/checkpoints/epoch=19-step=33000.ckpt"
params_path = "/home/jakob/Bachelorarbeit/GNNDensityGradients/lightning_logs/anova/default1/checkpoints/params.json"

# Best bayes network 32k
# checkpoint  = "/home/jakob/Bachelorarbeit/GNNDensityGradients/lightning_logs/bayes/best_bayes/checkpoints/epoch=31-step=32000.ckpt"
# params_path = "/home/jakob/ray_results4/LightningTrainer_2023-08-07_20-52-58/0e385726_32/params.json"

model = CConvModel.load_from_checkpoint(
    checkpoint_path = checkpoint,
    hparams         = load_hparams(params_path),
    device          = 'cuda'
)

model.eval()

density_data = DensityDataModule(
    data_dir    = 'datasets/data/dam_break_preprocessed/train',
    batch_size  = 1,
    data_split  = (1., 0., 0.),
    num_workers = 0, # Note that cuda only allows 0 workers.
    shuffle     = 0,
    cache       = False,       # Load dataset into memory
    device      = 'cuda',
)
density_data.setup("Initialize")
train_loader = density_data.train_dataloader()
train_iter = iter(train_loader)
index = 0

Setting up data module for stage  Initialize
Dataset is already preprepared. Not applying transform_once. (This means density & grads aren't recalculated.)


/home/jakob/anaconda3/envs/CConv/lib/python3.10/site-packages/torch/utils/data/dataset.py:342: UserWarning:

Length of split at index 1 is 0. This might result in an empty dataset.

/home/jakob/anaconda3/envs/CConv/lib/python3.10/site-packages/torch/utils/data/dataset.py:342: UserWarning:

Length of split at index 2 is 0. This might result in an empty dataset.



In [3]:
# to visualize the kernel, we need generate linspaced data
elem = next(train_iter)[0]
print("elem is element number", index)
index += 1

elem is element number 0


In [7]:
elem['density']

tensor([[ 0.3936],
        [ 1.2258],
        [ 1.2225],
        ...,
        [-0.9722],
        [-0.5265],
        [ 0.2985]], device='cuda:0')

In [12]:
for key in elem:
    print("Key", key, "shape", elem[key].shape)

Key pos shape torch.Size([3456, 3])
Key vel shape torch.Size([3456, 3])
Key m shape torch.Size([3456, 1])
Key viscosity shape torch.Size([3456, 1])
Key box shape torch.Size([10752, 3])
Key box_normals shape torch.Size([10752, 3])
Key density shape torch.Size([3456, 1])
Key temporal_density_gradient shape torch.Size([3456, 1])


In [10]:
import torch

def get_particles_with_dist(dist, vel_diff = 1.0):
    elem = {
        'pos': torch.tensor([[0.0, 0.0, 0.0], [dist, 0.0, 0.0]]),
        'vel': torch.tensor([[vel_diff / 2., 0.0, 0.0], [- vel_diff / 2, 0.0, 0.0]]),
        'm': torch.tensor([0.125, 0.125]).view(-1, 1),
        'box': torch.tensor([[-100.0, -100.0, -100.0], [100.0, 100.0, 100.0]]),
        'box_normals': torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]]),
        'density': torch.tensor([0.8, 0.8]).view(-1, 1),
        # 'spatial_density_gradient': torch.tensor([[1.0, 1.0, 1.0], [1.0, 1.0, 1.0]]),
        # 'temporal_density_gradient': torch.tensor([[1.0, 1.0, 1.0], [1.0, 1.0, 1.0]])
    }

    for key in elem:
        # print("Key", key, "shape", elem[key].shape)
        elem[key] = elem[key].to('cuda')
        # print("Device", elem[key].device)

    return elem

# elem[key].shape


In [33]:
model(elem).cpu().detach().numpy()[0].item()

-0.23934829235076904

In [28]:
get_particles_with_dist(1.0)

{'pos': tensor([[0., 0., 0.],
         [1., 0., 0.]], device='cuda:0'),
 'vel': tensor([[0., 0., 0.],
         [0., 0., 0.]], device='cuda:0'),
 'm': tensor([[1.],
         [1.]], device='cuda:0'),
 'box': tensor([[-10., -10., -10.],
         [ 10.,  10.,  10.]], device='cuda:0'),
 'box_normals': tensor([[ 1.,  0.,  0.],
         [-1.,  0.,  0.]], device='cuda:0'),
 'density': tensor([[1.],
         [1.]], device='cuda:0'),
 'spatial_density_gradient': tensor([[1., 1., 1.],
         [1., 1., 1.]], device='cuda:0'),
 'temporal_density_gradient': tensor([[1., 1., 1.],
         [1., 1., 1.]], device='cuda:0')}

In [40]:
model(get_particles_with_dist(0.0))

tensor([[-0.1156],
        [-0.1156]], device='cuda:0', grad_fn=<MulBackward0>)

In [57]:
# linspace area with numpy
lin_space = torch.linspace(0, load_hparams(params_path)['filter_extent'], 100)
inputs = [get_particles_with_dist(d) for d in lin_space]
outs = [model(input) for input in inputs]
raw = [out.cpu().detach().numpy()[0].item() for out in outs]

In [58]:
px.line(x=lin_space, y=raw, title='Raw', labels={'x': 'x', 'y': 'y'})

In [60]:
elem

{'pos': tensor([[0., 0., 0.],
         [1., 0., 0.]], device='cuda:0'),
 'vel': tensor([[0., 0., 0.],
         [0., 0., 0.]], device='cuda:0'),
 'm': tensor([[1.],
         [1.]], device='cuda:0'),
 'box': tensor([[-10., -10., -10.],
         [ 10.,  10.,  10.]], device='cuda:0'),
 'box_normals': tensor([[ 1.,  0.,  0.],
         [-1.,  0.,  0.]], device='cuda:0'),
 'density': tensor([[1.],
         [1.]], device='cuda:0'),
 'spatial_density_gradient': tensor([[1., 1., 1.],
         [1., 1., 1.]], device='cuda:0'),
 'temporal_density_gradient': tensor([[1., 1., 1.],
         [1., 1., 1.]], device='cuda:0')}

In [80]:
transformations = [AddTemporalDensityGradient(include_box=False)]
# transformations += [NormalizeDensityData()]
transformations = Compose(transformations)
inputs       = [get_particles_with_dist(d) for d in lin_space]
inputs       = [transformations(input) for input in inputs]
inputs[0]

{'pos': tensor([[0., 0., 0.],
         [0., 0., 0.]], device='cuda:0'),
 'vel': tensor([[0., 0., 0.],
         [0., 0., 0.]], device='cuda:0'),
 'm': tensor([[0.1250],
         [0.1250]], device='cuda:0'),
 'box': tensor([[-100., -100., -100.],
         [ 100.,  100.,  100.]], device='cuda:0'),
 'box_normals': tensor([[1., 0., 0.],
         [0., 1., 0.]], device='cuda:0'),
 'density': tensor([[0.8000],
         [0.8000]], device='cuda:0'),
 'temporal_density_gradient': tensor([[0.],
         [0.]], device='cuda:0')}

In [83]:
inputs[1]

{'pos': tensor([[0.0000, 0.0000, 0.0000],
         [0.0016, 0.0000, 0.0000]], device='cuda:0'),
 'vel': tensor([[0., 0., 0.],
         [0., 0., 0.]], device='cuda:0'),
 'm': tensor([[0.1250],
         [0.1250]], device='cuda:0'),
 'box': tensor([[-100., -100., -100.],
         [ 100.,  100.,  100.]], device='cuda:0'),
 'box_normals': tensor([[1., 0., 0.],
         [0., 1., 0.]], device='cuda:0'),
 'density': tensor([[0.8000],
         [0.8000]], device='cuda:0'),
 'temporal_density_gradient': tensor([[0.],
         [0.]], device='cuda:0')}

In [13]:
dataframe

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-0.185339,-0.242366,-0.306051,-0.375091,-0.443692,-0.513631,-0.578223,-0.641526,-0.696856,-0.746545,...,-0.163500,-0.163500,-0.163500,-0.163500,-0.163500,-0.163500,-0.163500,-0.163500,-0.163500,-0.163500
1,-0.121146,-0.164148,-0.206312,-0.249623,-0.294745,-0.336980,-0.377521,-0.415205,-0.451269,-0.486625,...,-0.238785,-0.238785,-0.238785,-0.238785,-0.238785,-0.238785,-0.238785,-0.238785,-0.238785,-0.238785


In [19]:
dataframe.T

,0,1
0,-0.185339,-0.121146
1,-0.242366,-0.164148
2,-0.306051,-0.206312
3,-0.375091,-0.249623
4,-0.443692,-0.294745
...,...,...
195,-0.163500,-0.238785
196,-0.163500,-0.238785
197,-0.163500,-0.238785
198,-0.163500,-0.238785


In [25]:
model_out = np.array([model_output, model_output2])
model_out.shape

(2, 200)

,x,Model Output,Model Output 2
0,0.000000,-0.185339,-0.121146
1,0.000789,-0.242366,-0.164148
2,0.001579,-0.306051,-0.206312
3,0.002368,-0.375091,-0.249623
4,0.003158,-0.443692,-0.294745
...,...,...,...
195,0.153940,-0.163500,-0.238785
196,0.154730,-0.163500,-0.238785
197,0.155519,-0.163500,-0.238785
198,0.156308,-0.163500,-0.238785


In [49]:
df.columns[1:]

Index(['Model Output', 'Model Output 2'], dtype='object')

In [50]:

fig = px.line(df, x='x', y=df.columns[1:], title='Model predictions')
fig.show()

In [60]:
px.line(df_target, x='x', y=df_target.columns[1:], title='Target values').show()

In [135]:
import torch
from utils.transforms import ToSample, AddDensity, AddTemporalDensityGradient, AddSpatialDensityGradient, NormalizeDensityData, ToNumpy
from torchvision.transforms import Compose
# transformations = [AddDensity(include_box=False)]
transformations = [AddTemporalDensityGradient(include_box=False)]
# transformations += [NormalizeDensityData()]
transformations = Compose(transformations)


def gauge_kernel_grad(model, values, vel_diff = 1.0, n = 200):
    inputs       = [get_particles_with_dist(d, vel_diff = vel_diff) for d in values]
    inputs       = [transformations(input) for input in inputs]
    outs         = [model(input) for input in inputs]
    upscale_factor = 236.4357 # 197.29
    model_output = [out.cpu().detach().numpy()[0].item() for out in outs]
    model_output = [x * upscale_factor for x in model_output]

    targets = [input['temporal_density_gradient'] for input in inputs]
    targets = [t.cpu().detach().numpy()[0].item() for t in targets]
    return model_output, targets


# labels = ['x', 'Model Output', 'Model Output 2']

number_of_lines = 10
filter_extent = load_hparams(params_path)['filter_extent'] # not used

lin_space   = torch.linspace(0, 0.12, 200)
x_vals      = [x.numpy().item() for x in lin_space]
data        = [x_vals]
data_target = [x_vals]
vel_diffs   = torch.linspace(-0.5, 1.0, number_of_lines)
labels      = ['x']
for vel_diff in vel_diffs:
    model_output, target = gauge_kernel_grad(model, lin_space, vel_diff = vel_diff, n = 200)
    data.append(model_output)
    data_target.append(target)
    labels.append(f'{vel_diff:.2f}')

df = pd.DataFrame(data, index=labels).T
df_target = pd.DataFrame(data_target, index=labels).T

In [121]:
poly6_reverse(-0.5)

(1.1701797540132999-0.23311026195538678j)

In [122]:
def poly6_reverse(x):
    if x < 0:
        return -poly6_reverse(-x)
    return np.sqrt(1-x**(2./3))

In [80]:
print(fig_target.layout.height)

None


In [115]:
fig_model.write_image(dir + "search3_kernel_learned_default.pdf", width=1400, height=455, scale=2.8)


In [136]:
pio.templates.default = "simple_white"
# pio.templates.default = "plotly_white"

fig_model = px.line(df, x='x', y=df.columns[1:]) # title='Model predictions'
fig_model.update_layout(legend_title_text='Δv', xaxis_title='x', yaxis_title='Δρ/Δt')
fig_model.update_layout(margin=dict(l=20, r=20, t=0, b=0), font = dict(family = "serif", size = 24))
fig_model.update_layout(width=1400, height = 455)
fig_model.update_traces(line=dict(width=2.5))
fig_model.update_layout(yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 100
    ))
# set line width
# for d in fig_model.data:
#     d.line.width = 2.5
fig_model.show()

fig_target = px.line(df_target, x='x', y=df_target.columns[1:]) # title='Target values'
fig_target.update_layout(legend_title_text='Δv', xaxis_title='x', yaxis_title='Δρ/Δt')
fig_target.update_layout(margin=dict(l=20, r=20, t=0, b=0), font = dict(family = "serif", size = 24))
fig_target.update_layout(width=1400, height = 455)
fig_target.update_traces(line=dict(width=2.5))
fig_target.update_yaxes(range=fig_model.layout.yaxis.range)
fig_target.update_layout(yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 100
    ))
# for d in fig_target.data:
#     d.line.width = 2.5
fig_target.show()

dir = "/home/jakob/Bachelorarbeit/GNNDensityGradientsThesis/figures/"
# fig_model.write_image(dir + "search3_kernel_learned.pdf", width=1400, height=455, scale=2.8)
# fig_model.write_image(dir + "search3_kernel_learned.pdf", width=1400, height=455, scale=2.8)
# fig_target.write_image(dir + "search3_kernel_target.pdf", width=1400, height=455, scale=2.8)
# fig_target.write_image(dir + "search3_kernel_target.pdf", width=1400, height=455, scale=2.8)
# fig_target.write_image(dir + "search3_kernel_target.pdf", width=1400, height=455, scale=3)

In [51]:
from utils.visualization import *
pio.templates.default = "plotly"
for _ in range(1):
    fig = plot_sample_for_tensorboard(model, density_data.dataset['train'])

    # remove legend and make borders smaller
    fig.update_layout(showlegend=False, margin=dict(l=0, r=0, b=0, t=0, pad=0))
    fig.update_layout(width=1250, height=400)
    fig.show()

In [17]:
elem['vel']

tensor([[-4.0019e-08,  0.0000e+00,  4.0670e-02],
        [-2.1491e-01,  0.0000e+00,  2.4677e-02],
        [ 1.7575e-08,  0.0000e+00, -2.2260e-03],
        ...,
        [-4.9842e-01, -2.8507e-01, -3.3422e-01],
        [-7.8207e-01, -1.7016e-01, -2.2302e-01],
        [-4.9665e-01, -3.4500e-01, -8.8318e-02]], device='cuda:0')

In [29]:
# elem.pop('vel')
# plot_sample(elem)
fig = plot_particles(elem['pos'], elem['density'], colorscale='Plasma', return_fig=True)
# remove margins
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.update_layout(scene_camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)))
fig.show()
# zoom camera out
